# 环境配置

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from tqdm import trange

# The multivariate Variance Gamma model basket option pricing and calibration 论文复刻

Parameter  
$r = 3\%$  
$\rho_{i,j}=0, ~for~i \ne j ~and~ i, j = 1, 2, 3$  
$N = 10^6$

| 1| 1 | 2 | 3|
|- |- |- |- |
|$\mu$|-0.1368 | -0.056 | -0.1984 |
|$\sigma$| 0.1099 | 0.1677 | 0.0365 |
| $X(0)$ | 100 | 200 | 300 |
| $\omega$ | 1/3 | 1/6 | 1/9 |

$$\varepsilon[K]=\frac{|\overline{C}[K] - C^{sim}[K]|}{C^{sim}[K]}$$

## table1

In [2]:
r = 0.03
rho = np.eye(3)
X_0 = np.array([100, 200, 300])
q = np.array([0, 0, 0])
omega = np.array([0, 0, 0])
mu = np.array([-0.1368, -0.056, -0.1984])
sigma = np.array([0.1099, 0.1677, 0.0365])
weight = np.array([1/3, 1/6, 1/9]) # w是权重，omega才是参数
K = np.array([0, 0, 0])
parameter_list = [X_0, q, omega, mu, sigma, weight, K]

table1 = pd.DataFrame(data =  {'X_0': X_0, 'mu': mu,'sigma': sigma, 'weight' : weight},
                      index =  ['stock 1', 'stock 2', 'stock 3'])
table1

,X_0,mu,sigma,weight
stock 1,100,-0.1368,0.1099,0.333333
stock 2,200,-0.0560,0.1677,0.166667
stock 3,300,-0.1984,0.0365,0.111111


## table2

先做个表头

In [2]:
from itertools import product
# T, v, K, C_hat, C_sim, vare

T = [1, 2]
v = [0.5, 0.75, 0.9]
K = [75, 90, 100, 110, 125]

table2 = pd.DataFrame(list(product(T, v, K)), columns = ['T', 'v', 'K']).set_index(['T','v', 'K'])
table2.loc[:,['$\overline{C}[K]$','$C^sim[K]$', r'$\varepsilon[K]$']] = 0
table2.T

T                    1                                               ...    2  \
v                 0.50                     0.75                      ... 0.75   
K                  75   90   100  110  125  75   90   100  110  125  ...  75    
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

T                                                               
v                                     0.90                      
K                  90   100  110  125  75   90   100  110  125  
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3 rows x 30 columns]

### 计算所有T, v, K

In [3]:
from scipy.stats import norm
import scipy.integrate as sci
import math
from scipy.stats import norm
from scipy.optimize import fsolve # 解决非线性方程组的数值求解问题
from scipy.optimize import bisect # 解决非线性方程组的数值求解问题
from scipy.stats import gamma # gamma分布的概率密度

#### upper

3

In [13]:
def C_upper_calculate_func(T, v, K, sigma, mu, omega, r, q):
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(omega)
    def f(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x)/(sigma * np.sqrt(x))
        d_2 = d_1 - sigma * np.sqrt(x)
        exp_upper = np.exp((r - q + omega ) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_upper * norm.cdf(d_1) - K_i * norm.cdf(d_2)))
        S_price_numerical = np.dot(X_price_numerical, weight)
        return S_price_numerical * gamma_density * np.exp(-r * T)
    C_upper = sci.quad(f, 0, np.inf)[0]
    return C_upper

#### lower

$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

In [15]:
def C_lower_calculate_func(T, v, K, sigma, mu, omega, r, q):
    def g(x):
        omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
        K_i = K / weight.copy() / len(omega)
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/sigma2Lambda
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x / 2 * (1 + r_i ** 2))/(sigma * np.sqrt(x) * r_i)
        d_2 = d_1 - sigma * np.sqrt(x) * r_i
        exp_lower = np.exp((r + omega - q) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_lower * norm.cdf(d_1) - K_i * norm.cdf(d_2)))
        S_price_numerical = np.dot(X_price_numerical, weight)
        return S_price_numerical * gamma_density * np.exp(-r * T)
    C_lower = sci.quad(g, 0, np.inf)[0]
    return C_lower

#### z_y to approximate

##### var upper

\begin{align}
& \operatorname{Var}\left[S_y^c\right]=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) \\
& \quad e^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j\right) y+\frac{\sigma_i^2+\sigma_j^2}{2} y}\left(e^{\sigma_i \sigma_j y}-1\right) .
\end{align}

In [6]:
def var_upper_calculate_func(T, v, K, sigma, mu, omega, r, q):
    def var_upper_calculate(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_upper_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_upper_part2 = np.exp(2 * r * T)
        var_upper_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_upper_part4 = np.exp(np.outer(sigma,sigma) * x) - 1
        var_upper_int = sum(sum(var_upper_part1 * var_upper_part2 * var_upper_part3 * var_upper_part4))
        return var_upper_int * gamma_density
    var_upper = sci.quad(var_upper_calculate, 0, np.inf)[0]
    return var_upper

\begin{align*}
\operatorname{Var}  {\left[S_y^l\right]}&={\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
&\quad \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2\left(1-r_i^2\right)+\sigma_j^2\left(1-r_j^2\right)\right)\right) y} \\
&\quad \times e^{\frac{1}{2}(r^2_i\sigma^2_i+r^2_j\sigma^2_j)y}(e^{r_ir_j\sigma_i\sigma_jy}-1)\\
& {=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
&\quad \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2+\sigma_j^2\right)\right) y} \\
& \quad\times (e^{r_ir_j\sigma_i\sigma_jy}-1)
\end{align*}

权重是$w$  
$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

##### var lower 

In [7]:
def var_lower_calculate_func(T, v, K, sigma, mu, omega, r, q):
    def var_lower_calculation(x):
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/np.sqrt(sigma2Lambda)
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_lower_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_lower_part2 = np.exp(2 * r * T)
        var_lower_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_lower_part4 = np.exp(np.outer(sigma * r_i, sigma * r_i) * x) - 1
        var_lower_int = sum(sum(var_lower_part1 * var_lower_part2 * var_lower_part3 * var_lower_part4))
        return var_lower_int * gamma_density
    var_lower = sci.quad(var_lower_calculation, 0, np.inf)[0]
    return var_lower

##### var Sy

\begin{align*}
	\mathrm{Var}[{_y}S]=\sum_{i=1}^{n} \sum_{j=1}^{n} w_{i} w_{j} X_{i}(0) X_{j}(0)  
	\mathrm{e}^{2 r T+\left(\omega_{i}-q_{i}+\omega_{j}-q_{j}\right) T+\left(\mu_{i}+\mu_{j}\right) y} 
	e^{\frac{\sigma_i^2+\sigma_j^2+\rho_{i,j}\sigma_i\sigma_j
	}{4-\rho^2_{i,j}}2(1-\rho^2_{i,j})y}
\end{align*}


In [8]:
def var_Sy_calculate_func(T, v, K, sigma, mu, omega, r, q):
    def var_calculation(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_part2 = np.exp(2 * r * T)
        var_part3 = np.outer(np.exp((weight - q) * T + mu * x),np.exp((weight - q) * T + mu * x))
        var_part4_i_ne_j = np.exp((sigma ** 2 + (sigma ** 2).reshape(len(sigma),-1) + np.outer(sigma, sigma) * rho) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        np.fill_diagonal(var_part4_i_ne_j, np.outer(np.exp(x * sigma ** 2), np.exp(x * sigma ** 2 * 2))) # 生成E[XY] # TODO
        var_part4 = var_part4_i_ne_j - np.outer(np.exp(sigma ** 2 / 2 * x), np.exp(sigma ** 2 / 2 * x))
        # var_part4 = (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) * (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) #* np.exp(np.outer(sigma,sigma) * rho  * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        var_int = sum(sum(var_part1 * var_part2 * var_part3 * var_part4))
        return var_int * gamma_density
    var_Sy = sci.quad(var_calculation, 0, np.inf)[0]
    return var_Sy

$$z_y = \frac{Var[S_y^c]-Var[S_y]}{Var[S_y^c]-Var[S_y^l]}$$  

#### sim

##### 生成随机数

##### 蒙特卡洛（向量化，指令并行）

In [14]:
from tqdm import tqdm
def monte_carlo_calculate_func(T, v, K, sigma, mu, omega, r, q, power_num):
    sim_times = 10 ** power_num
    np.random.seed(20240321) 
    Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
    Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(omega)
    X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
    S_price_vector = np.dot(X_price_vector, weight)
    C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
    C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
    return C_sim

### 带入参数

In [10]:
r = 0.03
rho = np.eye(3)
X_0 = np.array([100, 200, 300])
q = np.array([0, 0, 0])
# omega = np.array([0, 0, 0])
mu = np.array([-0.1368, -0.056, -0.1984])
sigma = np.array([0.1099, 0.1677, 0.0365])
weight = np.array([1/3, 1/6, 1/9]) # w是权重，omega才是参数
# K = np.array([0, 0, 0])

In [17]:
from tqdm import tqdm
C_sim_result_list = list()
C_approximate_list = list()
power_num = 6

for (T,v,K) in tqdm(table2.index.values):
    C_upper = C_upper_calculate_func(T, v, K, sigma, mu, omega, r, q)
    C_lower = C_lower_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_upper = var_upper_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_lower = var_lower_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_Sy = var_Sy_calculate_func(T, v, K, sigma, mu, omega, r, q)
    C_sim = monte_carlo_calculate_func(T, v, K, sigma, mu, omega, r, q, power_num)
    C_sim_result_list.append(C_sim)
    z_y = (var_upper - var_Sy)/(var_upper - var_lower)
    C_approximate = z_y * C_lower + (1-z_y) * C_upper
    C_approximate_list.append(C_approximate)

100%|██████████| 30/30 [00:28<00:00,  1.04it/s]


In [19]:
table2.loc[:,'$\overline{C}[K]$'] = C_approximate_list
table2.loc[:,'$C^sim[K]$'] = C_sim_result_list
table2.loc[:,r'$\varepsilon[K]$'] = abs(table2.loc[:,'$\overline{C}[K]$'] - table2.loc[:,'$C^sim[K]$'])/ C_sim_result_list
table2 = table2.round(4)
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$
T v    K                                                   
1 0.50 75             27.4592     27.3116            0.0054
       90             14.0236     13.5774            0.0329
       100             6.5495      5.9578            0.0993
       110             1.9875      1.2027            0.6525
       125             0.2386      0.0060           38.5718
  0.75 75             27.6455     27.4386            0.0075
       90             14.4330     13.9589            0.0340
       100             7.0179      6.4220            0.0928
       110             2.2336      1.3957            0.6004
       125             0.2271      0.0051           43.3726
  0.90 75             27.7609     27.5319            0.0083
       90             14.6454     14.1706            0.0335
       100             7.2420      6.6586            0.0876
       110             2.3470      1.4925            0.5725
       125             0.2212      0.0048           44.6569
2 0.50 75             29.9077     29.6091            0.0101
       90             17.3893     16.6890            0.0420
       100            10.3636      9.4982            0.0911
       110             5.2106      4.2383            0.2294
       125             1.5239      0.5537            1.7520
  0.75 75             30.2232     29.8354            0.0130
       90             17.9822     17.2421            0.0429
       100            11.0774     10.1977            0.0863
       110             5.8540      4.8520            0.2065
       125             1.7945      0.7026            1.5541
  0.90 75             30.4119     29.9823            0.0143
       90             18.2933     17.5383            0.0431
       100            11.4318     10.5484            0.0837
       110             6.1666      5.1560            0.1960
       125             1.9339      0.7811            1.4757

## figure 1

$$f_{\bar{S}}(K)=\mathrm{e}^{r T} \frac{\partial \bar{C}^2[K]}{\partial K^2}$$

In [57]:
import torch
T = 1
v = 0.5
K = 75
r = 0.03
rho = np.eye(3)
X_0 = np.array([100, 200, 300])
q = np.array([0, 0, 0])
omega = np.array([0, 0, 0])
mu = np.array([-0.1368, -0.056, -0.1984])
sigma = np.array([0.1099, 0.1677, 0.0365])
weight = np.array([1/3, 1/6, 1/9])

sim_times = 10 ** power_num
np.random.seed(20240321) 
Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
K_i = K / weight.copy() / len(omega)
X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
S_price_vector = np.dot(X_price_vector, weight)
C_sim = np.dot((S_price_vector - K), (S_price_vector - K > 0)) / len(S_price_vector)* np.exp(-r * T)
# C_sim.backward()
# K.grad
C_sim


27.29809346946704

In [22]:
import torch

# 创建一个需要求导的张量
x = torch.tensor(2.0, requires_grad=True)
x_i = x / torch.tensor([2.0,2.0], requires_grad=True)

# 定义一个函数 y = x^2
y = sum(x_i ** 2 + x_i)

# 对 y 进行求导
y.backward()

# 打印导数值
print(x.grad)  # 输出：tensor(4.)


tensor(3.)


# bak

## 换逻辑前，list然后append然后整体运算

In [ ]:
from tqdm import tqdm
C_upper_lst = list()
C_lower_lst = list()
var_upper_lst = list()
var_lower_lst = list()
var_Sy_lst = list()
C_sim_list = list()
C_sim_result_list = list()
power_num = 6

for (T,v,K) in tqdm(table2.index.values):
    C_upper = C_upper_calculate_func(T, v, K, sigma, mu, omega, r, q)
    C_upper_lst.append(C_upper)
    C_lower = C_lower_calculate_func(T, v, K, sigma, mu, omega, r, q)
    C_lower_lst.append(C_lower)
    var_upper = var_upper_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_upper_lst.append(var_upper)
    var_lower = var_lower_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_lower_lst.append(var_lower)
    var_Sy = var_Sy_calculate_func(T, v, K, sigma, mu, omega, r, q)
    var_Sy_lst.append(var_Sy)
    C_sim = monte_carlo_calculate_func(T, v, K, sigma, mu, omega, r, q, power_num)
    C_sim_result_list.append(C_sim)
    z_y = (np.array(var_upper_lst) - np.array(var_Sy_lst))/(np.array(var_upper_lst) - np.array(var_lower_lst))
    C_approximate = z_y * C_lower_lst + (1-z_y) * C_upper_lst
    
table2.loc[:,'$\overline{C}[K]$'] = C_approximate
table2.loc[:,'$C^sim[K]$'] = C_sim_result_list
table2.loc[:,r'$\varepsilon[K]$'] = abs(table2.loc[:,'$\overline{C}[K]$'] - table2.loc[:,'$C^sim[K]$'])/ table2.loc[:,'$C^sim[K]$']
table2 = table2.round(4)
table2
# table2.loc[:,'$C^u$'] = C_upper_lst
# table2.loc[:,'$C^l$'] = C_lower_lst
# table2.loc[:,'$Var^u$'] = var_upper_lst
# table2.loc[:,'$Var^l$'] = var_lower_lst
# table2.loc[:,'$Var[Sy]$'] = var_Sy_lst
# table2.loc[:,'$z_y$'] = (table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var[Sy]$'])/(table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var^l$'] )
# table2.loc[:,'$\overline{C}[K]$'] = table2.loc[:,'$z_y$'] * table2.loc[:,'$C^l$'] + (1-table2.loc[:,'$z_y$']) * table2.loc[:,'$C^u$']